In [315]:
%load_ext autoreload
%autoreload 2

from spark.preprocessor import load_q_data, load_timeseries_data, train_test_split_ids, Preprocess_Q, Preprocess_Time_Boss
from spark.preprocessor import preprocessor_reduced
from spark.model_io import save_model_combined

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import  StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.metrics import balanced_accuracy_score

from catboost import CatBoostClassifier
from xgboost import XGBClassifier

import joblib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [298]:
raw_prep_dir  = '../processed_data/'

In [299]:
X_train_q, X_test_q, X_train_ids, X_test_ids, y_train, y_test = preprocessor_reduced(raw_prep_dir + 'merged_dfq.csv')

✅ Transformer saved at: ./model_registry/transformer_20250910-210501.joblib


In [300]:
X_test_ids

294    379
115    405
45     330
250     31
16      15
      ... 
395    172
176    300
77     448
373    294
7       54
Name: id, Length: 94, dtype: int64

In [301]:
X_train_q

array([[-0.41176471, -0.23529412,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.29411765, -0.47058824,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.52941176,  0.11764706,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.94117647,  0.52941176,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.05882353, -0.47058824,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.47058824, -0.76470588,  0.        , ...,  1.        ,
         0.        ,  0.        ]], shape=(375, 10))

In [302]:
time_data = load_timeseries_data('../processed_data/')

In [303]:
time_data

<xarray.DataArray 'timeseries' (id: 469, channel: 132, timestep: 976)> Size: 242MB
array([[[-7.18307542e-03, -7.97853246e-03, -7.81304389e-03, ...,
          1.72191102e-03,  2.75583961e-03,  1.86453806e-03],
        [-2.89946591e-04, -1.60217297e-03, -2.94661568e-03, ...,
          2.54398095e-04,  1.34622052e-04,  1.96311320e-03],
        [ 4.12839325e-03,  3.16892145e-03,  2.27304758e-03, ...,
          4.87453333e-04,  1.48784160e-03, -4.89200873e-04],
        ...,
        [ 7.97079727e-02,  7.00292438e-02,  4.42504846e-02, ...,
         -1.05950013e-02, -6.27173204e-03,  3.40433232e-03],
        [-1.02272712e-01, -9.49295759e-02, -7.82209039e-02, ...,
          6.99309483e-02,  6.35627955e-02,  5.61850406e-02],
        [-1.67727657e-02, -1.67666413e-02, -2.31204107e-02, ...,
         -1.13539649e-02, -1.77311711e-02, -2.30463073e-02]],

       [[ 8.36125482e-03,  5.90733485e-03, -4.05572727e-03, ...,
         -1.88475254e-03, -9.19213693e-04,  9.88525455e-04],
        [-1.96502102e-03,  2.39871955e-03, -4.03387845e-03, ...,
         -1.77135039e-02, -1.80895422e-02, -1.35900667e-02],
        [ 1.00842845e-02,  1.44647313e-02,  1.11305481e-02, ...,
         -3.04453704e-03, -3.19689699e-03, -2.37555546e-03],
...
         -1.08799472e-01, -9.72944349e-02, -6.82388172e-02],
        [-4.84612212e-02, -2.36799736e-02,  5.07843681e-03, ...,
          1.09375022e-01,  1.36836991e-01,  1.55176848e-01],
        [ 1.09312020e-01,  1.07321836e-01,  1.17433533e-01, ...,
         -4.83003780e-02, -8.27455670e-02, -1.17528580e-01]],

       [[ 1.71402353e-03,  5.86650684e-04,  1.42322108e-03, ...,
         -2.10232858e-04, -3.14288121e-03, -4.14645532e-03],
        [ 4.81363339e-03,  3.89338937e-03,  2.92152795e-03, ...,
          6.13220152e-04,  1.58190704e-03,  6.21251180e-04],
        [ 4.43531526e-03,  3.40049202e-03,  3.71357426e-04, ...,
          1.04908447e-03, -2.86440458e-03, -1.89287565e-03],
        ...,
        [-2.55793445e-02, -3.29656228e-02, -5.21528497e-02, ...,
         -1.21610969e-01, -1.19265735e-01, -1.17108569e-01],
        [-1.36809602e-01, -1.45383656e-01, -1.47666246e-01, ...,
         -1.36324510e-01, -1.57518074e-01, -1.59600675e-01],
        [-1.18657626e-01, -1.21845432e-01, -1.20780163e-01, ...,
         -7.82845020e-02, -7.72348121e-02, -7.40510076e-02]]],
      shape=(469, 132, 976), dtype=float32)
Coordinates:
  * id        (id) int64 4kB 1 2 3 4 5 6 7 8 ... 462 463 464 465 466 467 468 469
  * channel   (channel) <U38 20kB 'Relaxed1_Acceleration_LeftWrist_X' ... 'En...
  * timestep  (timestep) int64 8kB 0 1 2 3 4 5 6 ... 969 970 971 972 973 974 975

In [304]:
X_train_time = time_data.sel(id = list(X_train_ids))
X_test_time = time_data.sel(id = list(X_test_ids))
print(X_train_time.to_numpy().shape, X_train_q.shape)
X_test_q.shape

(375, 132, 976) (375, 10)


(94, 10)

In [305]:
X_train_time = X_train_time.to_numpy()
X_test_time = X_test_time.to_numpy()

In [306]:
X_test_ids.to_csv(raw_prep_dir + 'test_ids.csv')

In [307]:
boss = Preprocess_Time_Boss()

In [308]:
boss.fit(X_train_time)

BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(windo

In [310]:
X_train_boss = boss.transform(X_train_time)

1


In [277]:
X_test_boss = boss.transform (X_test_time)

1


In [279]:
X_combined_train = np.concatenate((X_train_boss, np.array(X_train_q)), axis = 1)
X_combined_test = np.concatenate((X_test_boss, np.array(X_test_q)), axis = 1)

print(X_combined_train.shape,X_train_boss.shape)

(375, 390862) (375, 390852)


In [311]:
pca_combined = PCA(n_components = 100, random_state = 23)
X_pca_combo_train = pca_combined.fit_transform(X= X_combined_train)
X_pca_combo_test = pca_combined.transform(X=X_combined_test)

In [312]:
pd.DataFrame(X_pca_combo_test).to_csv("../processed_data/processed_data_combined_model_3.csv")

In [313]:
sv_model_pca_combined = SVC(kernel = 'rbf', class_weight = 'balanced', C = 1, gamma = 'scale')
sv_model_pca_combined.fit(X_pca_combo_train, y_train)

,C,1
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [327]:
y_pred_pca_combo = sv_model_pca_combined.predict(X_pca_combo_test)
balanced_accuracy_score(y_test, y_pred_pca_combo)

0.6682312252964427

In [317]:
cv_scores = cross_validate(
    sv_model_pca_combined, X_pca_combo_train, y_train,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='balanced_accuracy'
)

In [318]:
cv_scores

{'fit_time': array([0.00583124, 0.00521922, 0.00460601, 0.00446391, 0.00465298]),
 'score_time': array([0.00335479, 0.00289369, 0.00296807, 0.00260806, 0.00257516]),
 'test_score': array([0.61296296, 0.60910386, 0.58009583, 0.62088837, 0.6325093 ])}

In [319]:
param_grid = [
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto']},
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['poly'], 'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto'], 'degree': [2, 3, 4, 5]},
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto']}
]

In [320]:
search = GridSearchCV(
    sv_model_pca_combined,
    param_grid= param_grid,
    scoring='balanced_accuracy',
    cv=5, n_jobs=-1
)

In [321]:
search.fit(X_pca_combo_train, y_train)

,estimator,"SVC(C=1, clas...ht='balanced')"
,param_grid,"[{'C': [0.1, 1, ...], 'kernel': ['linear']}, {'C': [0.1, 1, ...], 'gamma': [0.001, 0.01, ...], 'kernel': ['rbf']}, ...]"
,scoring,'balanced_accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,1


In [322]:
search.best_estimator_

,C,1
,kernel,'sigmoid'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [323]:
y_pred_grid = search.predict(X_pca_combo_test)

In [196]:
balanced_accuracy_score(y_test, y_pred_grid)


0.6806324110671936

In [324]:
cv_scores = cross_validate(
    search, X_pca_combo_train, y_train,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='balanced_accuracy'
)

In [325]:
cv_scores['test_score'].mean()

np.float64(0.5845386387491651)

In [326]:
sv_model_params = search.best_params_
sv_model_params

{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}

In [293]:
#save_model_combined(search)

In [294]:
quest_data = pd.read_csv(raw_prep_dir + 'merged_dfq.csv')

In [ ]:
columns_to_keep = ['id', 'age_at_diagnosis', 'age',
                       'gender', 'appearance_in_kinship',
                       '02', '03', '09', '13', '17', '20','label']
quest_data_test = quest_data.copy()[quest_data.id.isin(X_test_ids)]
quest_data_test_tokeep = quest_data_test[columns_to_keep].reset_index()
quest_data_test_tokeep.drop(columns = 'index', inplace = True)
quest_data_test_tokeep

,id,age_at_diagnosis,age,gender,appearance_in_kinship,02,03,09,13,17,20,label
0,146,68,71,male,True,True,False,True,False,False,False,1
1,54,40,40,female,True,False,False,False,False,False,False,0
2,412,43,47,female,False,True,False,False,True,True,True,1
3,15,59,64,female,False,False,True,False,True,True,False,2
4,445,45,45,female,False,False,False,False,False,False,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...
89,283,61,61,female,False,False,False,False,False,False,False,0
90,41,56,64,male,False,False,True,True,True,False,True,1
91,313,56,63,male,False,True,False,True,False,False,True,1
92,169,51,56,female,True,False,False,True,False,False,False,1


In [ ]:
#quest_data_test_tokeep.to_csv('../processed_data/questionnaire_test.csv')